In [ ]:
#!sudo apt-get install -y python3-pymssql

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pymssql
import sys

In [ ]:
# pymssql documentation sample:
# http://www.pymssql.org/pymssql_examples.html
# from os import getenv
# import pymssql

# server = getenv("PYMSSQL_TEST_SERVER")
# user = getenv("PYMSSQL_TEST_USERNAME")
# password = getenv("PYMSSQL_TEST_PASSWORD")

# conn = pymssql.connect(server, user, password, "tempdb")
# cursor = conn.cursor()
# cursor.execute("""
# IF OBJECT_ID('persons', 'U') IS NOT NULL
#     DROP TABLE persons
# CREATE TABLE persons (
#     id INT NOT NULL,
#     name VARCHAR(100),
#     salesrep VARCHAR(100),
#     PRIMARY KEY(id)
# )
# """)
# cursor.executemany(
#     "INSERT INTO persons VALUES (%d, %s, %s)",
#     [(1, 'John Smith', 'John Doe'),
#      (2, 'Jane Doe', 'Joe Dog'),
#      (3, 'Mike T.', 'Sarah H.')])
# # you must call commit() to persist your data if you don't set autocommit to True
# conn.commit()

# cursor.execute('SELECT * FROM persons WHERE salesrep=%s', 'John Doe')
# row = cursor.fetchone()
# while row:
#     print("ID=%d, Name=%s" % (row[0], row[1]))
#     row = cursor.fetchone()

# conn.close()

In [ ]:
# sys.executable
# !which python
# sys.path

In [ ]:
server = "server:host"
user = "xx"
password = "xxxxxxx"
db = "xxxxx"

In [ ]:
conn = pymssql.connect(server, user, password, db)
cursor = conn.cursor()

In [ ]:
df = pd.read_csv('webtime-tracker-2022-06-19.csv')
# df = pd.read_csv('webtime-tracker-2022-06-14.csv')
# df = pd.read_csv('webtime-tracker-2022-01-15.csv')
# df = pd.read_csv('webtime-tracker-2021-09-15.csv')
# df = pd.read_csv('webtime-tracker-2021-08-08.csv')
# df = pd.read_csv('webtime-tracker-2021-06-06.csv')
# df = pd.read_csv('webtime-tracker-2021-01-17.csv')

In [ ]:
# df.head()

In [ ]:
df = df.melt(id_vars = ['Domain'],value_name='seconds', var_name='date' )

In [ ]:
df = df[df['seconds']>0]

In [ ]:
arr = df.to_numpy()

In [ ]:
######## use the pymssql internal executemany method to execute ########

In [ ]:
# command = []
# pre = "INSERT INTO webTracker VALUES (%s, %s, %s, %s, %s)" #domain, visit_date, visit_length, visit_from, record_insert_date
# for domain, date, length in arr: # there's a chance the there will be more than 1 columns for same date. named like date.1, date.2. We will only take the date part.
#     command.append( (domain, date.split('.')[0], str(length), location, currTime) )


In [ ]:
# %%time
# cursor.executemany(
#     pre,
#     command)

# CPU times: user 37.3 s, sys: 23.5 s, total: 1min
# Wall time: 19min 17s

In [ ]:
# conn.commit()

In [ ]:
############ Concatenate to a big sql and execute once (failed, as no sufficient memory) ############

In [ ]:
# head = "INSERT INTO webTracker VALUES"
# sql = ""
# for c in command:
#     sql  += head+ "('" + "',".join(c).replace("',","','")  + "');"
# #len(sql)

In [ ]:
# cursor.execute(sql)

In [ ]:
########## execute one command once ##########

In [ ]:
currTime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#location = 'UN_Laptop'
location = 'X1C6'
# location = 'T430'
pre = "INSERT INTO webTracker VALUES ('"
command = []
for domain, date, length in arr: # there's a change the there will be more than 1 columns for same date. named like date.1, date.2. We will only take the date part.
    command.append(pre+ domain + "','"  + date.split('.')[0] + "'," + str(length) + ",'" + location + "','" +  currTime + "')")

In [ ]:
#command

In [ ]:
# %%time
cnt = 0 
for c in command:
    cnt += 1 
    if cnt %1000 ==0: print(cnt)
    cursor.execute(c)
# CPU times: user 15.4 s, sys: 23.5 s, total: 38.8 s
# Wall time: 19min 50s

In [ ]:
conn.commit()

In [ ]:
cursor.execute("SELECT cnt = count(*) FROM webTracker WHERE record_insert_date= %s", currTime)

for row in cursor:
    print('Inserted %s records' %row[0])


In [ ]:
conn.close()